# Data Collection

Collecting data for the both the initial goal of clustering Redditors but also for the next step of the process, which is predicting game sales based on Reddit Submissions, Comments, and Users.

**Steps**
1. Create a list of games released on Steam that we are interested in.
2. Get release dates listed from Steam API.
3. Get submissisions about game on reddit, pre and post launch within a 5 day range. (5 is arbitrary)
3. Get comments from each submission.
4. Get reddit user information of authors of submissions and comments.

## Table of Contents
- [Steam](#Steam-Game-List-&-Data)
- [Reddit](#Reddit-Data) 

**Imports**

In [1]:
import configparser
import datetime as dt
import time

import numpy as np
import pandas as pd
import praw
import pymysql
import requests

from prawcore.exceptions import (Forbidden as PrawForbiddenError,
                                 NotFound as PrawNotFound)

from aws_mysql import MySqlConn

**Functions**

In [2]:
# TODO
def retry():
    """
    Retry api call on failure.
    """
    pass

# Steam ---------------------------------------------------------------------
def get_steam_app_id(name, app_ids):
    """
    Gets game Steam app id.
    :param: String, steam game name.
    :param app_ids: List, of dictionaries with Steam app name and id (From 
    Steam API)
    :return: String.
    """
    for app in app_ids:
        # TODO: implement fuzzy matching here
        if name.lower() == app['name'].lower():
            return str(app['appid'])

def get_game_data(steam_app_id, url):
    """
    Gets steam data of game from Steam API.
    :param steam_app_id: Int, Steam app id.
    :param url: Steam app url for getting app data.
    :return: List of dictionaries.
    """
    res = requests.get(url + steam_app_id)  
    if res.status_code != 200:
        print(f'Warning: {name} | {res.status_code}')
    return res.json()[steam_app_id]['data']

def get_dates(release_date, spread=5):
    """
    Calculates the initial and end days of range spread, before and
    after the release date, does not include the release date.
    :param release_date: DateTime, release date of game.
    :param spread: Number of days for the date range.
    :return: Tuple of four dates (epoch time, int) in order:
        Pre Launch Start Date, 
        Pre Launch End Date
        Post Launch Start Date
        Post Launch End Date
    """
    # pushshift api expects date param to be an int
    return (
        int((release_date - dt.timedelta(days=spread)).timestamp()),
        int((release_date - dt.timedelta(seconds=1)).timestamp()),
        int((release_date + dt.timedelta(days=1)).timestamp()),
        int((release_date + dt.timedelta(days=spread + 1)).timestamp())
    )

# Reddit --------------------------------------------------------------------
def get_submissions(search_term, start_epoch, end_epoch, sleep=1):
    """
    Uses Pushshift Reddit API to get all submissions for a search term within
    the provided date range.
    :param search_term: String used as search term in API call.
    :param start_epoch: Beginning date, int, to look for submissions.
    :param end_epoch: End date, int, to look for submissions.
    :param sleep: Time(seconds) to wait before running next API call. Defaults
    to 1 second.
    :return: Pandas DataFrame
    """
    params = {
        # q is more exhaustive than using title and selftext
        'q': search_term,  
#         'title': search_term,
#         'selftext': search_term,
        'after': start_epoch,
        'before': end_epoch,
        'size': 100  # 100 is API limit
    }
    
    prev_size = -1  # to check if there are results
    all_submissions = []
    while len(all_submissions) > prev_size:
        prev_size = len(all_submissions)
        res = requests.get(
            'https://api.pushshift.io/reddit/search/submission',
            params
        )
        
        if res.status_code == 200:
            submissions = res.json()['data']
            # TODO:
            # depending on what sbumissions returns when empty, can replace len(all_sub) > above
            if submissions:  
                all_submissions += submissions
                # do not want duplicate data
                params['after'] = max([submission['created_utc'] 
                                       for submission in submissions])
        time.sleep(sleep)
    df_ = pd.DataFrame(all_submissions)
    df_['search_term'] = search_term
    return df_, 

def get_comments(submission_id):
    """
    Gets all the comments of a submission and returns comment data and author 
    objects, (Redditor Instances).
    :param submission_id: String, id of reddit submission.
    :return: Tuple (List of data, List of Authors as Redditor Objects)
    """
    comments = []
    authors = []
    # https://praw.readthedocs.io/en/latest/tutorials/comments.html
    # writing over id but following documentation
    submission = reddit.submission(id=submission_id)  
    submission.comments.replace_more(limit=None)  # grab all comments
    for comment in submission.comments.list():
        print(len(comments))
        try:
            comments.append((
                comment.id,
                comment.author.id if comment.author else None,
                comment.body,
                comment.created_utc,
                comment.distinguished,
                comment.edited,
                comment.is_submitter,
                comment.link_id,
                comment.parent_id, 
                comment.score,
                comment.stickied,
                comment.subreddit_id
            ))

            authors.append(comment.author)
        except AttributeError as e:  # attribute error is from comments.author.id, prob if author is deleted
            comments.append((
                comment.id,
                None,
                comment.body,
                comment.created_utc,
                comment.distinguished,
                comment.edited,
                comment.is_submitter,
                comment.link_id,
                comment.parent_id, 
                comment.score,
                comment.stickied,
                comment.subreddit_id
            ))
        except (PrawForbiddenError, PrawNotFound):
            pass
            
    return comments, authors
    
def save_comments_and_authors(submission_ids):
    """
    Saves comment and author data into AWS rds.
    :param submission_ids: List/array of reddit submission ids.
    :return: None
    """
    db = MySqlConn(ENDPOINT, USER, PASSWORD, DBNAME)
    
    for submission_id in submission_ids:
        try:  # catch private submissions that may cause forbidden errors
            comments, authors = get_comments(submission_id)
            comments_sql = """
                INSERT INTO comments (CommentID, 
                                      AuthorID, 
                                      Body, 
                                      CreatedUTC, 
                                      Distinguished, 
                                      Edited, 
                                      IsSubmitter, 
                                      LinkID, 
                                      ParentID, 
                                      Score, 
                                      Stickied, 
                                      SubredditID) 
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
            """
            db.batch_insert(comments_sql, comments)
            comments = []

            users = {(author.id, 
                      author.comment_karma, 
                      author.created_utc, 
                      author.has_verified_email, 
                      author.is_employee, 
                      author.is_mod, 
                      author.is_gold, 
                      author.link_karma,
                      author.name) for author in authors if author}

            users_sql = """
                INSERT INTO users (UserID, 
                                   CommentKarma, 
                                   CreatedUTC, 
                                   VerifiedEmail, 
                                   IsEmployee, 
                                   IsMod, 
                                   IsGold, 
                                   LinkKarma, 
                                   Name) 
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
                ON DUPLICATE KEY UPDATE
                    UserID = UserID;
            """
            db.batch_insert(users_sql, users)
            authors = []

            print(f'Last updated submission id: ', submission_id)
            time.sleep(1)
        except PrawForbiddenError:
            pass

### Configuration

Pull in all necessary credentials to access AWS RDS and Praw.   
`config.ini` layout:  

```
[rds]  
endpoint: endpoint  
user: username
password: password
dbname: databasename


[praw]  
client_id: clientid  
client_secret: clientsecret  
user_agent: useragent  
```

In [3]:
config = config = configparser.ConfigParser()
config.read('./config.ini')

ENDPOINT = config['rds']['endpoint']
USER     = config['rds']['user']
PASSWORD = config['rds']['password']
DBNAME   = config['rds']['dbname']

CLIENTID     = config['praw']['client_id']
CLIENTSECRET = config['praw']['client_secret']
USERAGENT    = config['praw']['user_agent']

## Steam Game List & Data

List of games that are of interest. Games were chosen based on the following factors:
* Recency, post 2016
* Games without early access (release dates do not represent when game became available for play)
* Relatively successful games/well known games

In [4]:
# list of games for which to pull data
games = [
    "Fallout 4",
    "Cyberpunk 2077",
    "Stardew Valley",
    "Monster Hunter: World",
    "Sid Meier's Civilization IV",
    "Sid Meier's Civilization V",
    "Sid Meier's Civilization VI",
    "XCOM 2",
    "The Witcher 3: Wild Hunt",
    "Valheim",
    "DARK SOULS™ II",
    "DARK SOULS™ III",
    "Bioshock Infinite",
    "Total War: WARHAMMER",
    "Total War: WARHAMMER II",
    "Total War: THREE KINGDOMS",
    "Tropico 6",
    "Warhammer: Vermintide 2",
    "Red Dead Redemption 2",
    "Hades",
    "Bastion",
    "They are Billions",
    "Portal 2",
    "Doom",
    "Doom Eternal"
]

# Steam app list url and app url for game data
steam_app_list_url = 'https://api.steampowered.com/ISteamApps/GetAppList/v2/'
steam_app_url = 'https://store.steampowered.com/api/appdetails?appids='

# get list of Steam games and app ids
res = requests.get(steam_app_list_url)
steam_apps = res.json()['applist']['apps']

# get a list of steam app ids
steam_app_ids = [get_steam_app_id(game, steam_apps) for game in games]

# get Steam game data
game_data = [get_game_data(app_id, steam_app_url) 
             for app_id in steam_app_ids if app_id]

# want data in DataFrame, might use later
games_df = pd.DataFrame(game_data)
games_df.head()

,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,...,movies,recommendations,achievements,release_date,support_info,background,content_descriptors,demos,drm_notice,ext_user_account_notice
0,game,Fallout 4,377160,17,False,full,"[540810, 404090, 435881, 480631, 480630, 49065...","<h1>Review Highlights</h1><p><img src=""https:/...","Bethesda Game Studios, the award-winning creat...","Bethesda Game Studios, the award-winning creat...",...,"[{'id': 256658080, 'name': 'Fallout 4 Launch T...",{'total': 149272},"{'total': 84, 'highlighted': [{'name': 'War Ne...","{'coming_soon': False, 'date': 'Nov 9, 2015'}","{'url': 'http://help.bethsoft.com/', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",NaN,NaN,NaN
1,game,Cyberpunk 2077,1091500,18,False,NaN,NaN,<h1>Обратите внимание на другие игры от CD PRO...,"<img src=""https://cdn.akamai.steamstatic.com/s...","Cyberpunk 2077 — приключенческая ролевая игра,...",...,"[{'id': 256810260, 'name': '026_CP_Gameplay_Tr...",{'total': 371346},"{'total': 44, 'highlighted': [{'name': 'Шут', ...","{'coming_soon': False, 'date': '9 дек. 2020'}","{'url': 'http://en.cdprojektred.com/support', ...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [1, 2, 5], 'notes': 'Cyberpunk 2077 co...",NaN,NaN,NaN
2,game,Stardew Valley,413150,0,False,full,[440820],Stardew Valley is an open-ended country-life R...,Stardew Valley is an open-ended country-life R...,You've inherited your grandfather's old farm p...,...,"[{'id': 256660296, 'name': 'Stardew Valley Tra...",{'total': 308455},"{'total': 40, 'highlighted': [{'name': 'Greenh...","{'coming_soon': False, 'date': 'Feb 26, 2016'}","{'url': '', 'email': 'support@stardewvalley.net'}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",NaN,NaN,NaN
3,game,Monster Hunter: World,582010,0,False,NaN,"[1118010, 912540, 960781, 896580, 763695, 7636...","<h1>Featured DLC</h1><p><a href=""https://store...",Welcome to a new world! Take on the role of a ...,Welcome to a new world! In Monster Hunter: Wor...,...,"[{'id': 256769024, 'name': '191128_防衛隊派生武器_Eng...",{'total': 196308},"{'total': 100, 'highlighted': [{'name': 'Conqu...","{'coming_soon': False, 'date': 'Aug 9, 2018'}","{'url': 'http://www.capcom.com/mhwsupport/', '...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",NaN,NaN,NaN
4,game,Sid Meier's Civilization® IV,3900,0,False,NaN,NaN,With over 6 million units sold and unprecedent...,With over 6 million units sold and unprecedent...,With over 6 million units sold and unprecedent...,...,NaN,{'total': 1732},NaN,"{'coming_soon': False, 'date': 'Oct 25, 2006'}","{'url': 'http://support.2kgames.com/', 'email'...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",NaN,NaN,NaN


### Pre & Post Date Ranges

Need pre and post launch start and end dates as parameters for PushShift API to get submissions in a date range.

In [40]:
# api returns dictionary release_date
games_df['release_date'] = games_df['release_date'].map(lambda x: x['date'])
games_df['release_date'].str.replace('мая.', 'Mar', regex=False)  # temp solution: replace unknown strings
games_df['release_date'] = pd.to_datetime(games_df['release_date'])

# get the pre and post launch date ranges
games_df['launch_dates'] = games_df['release_date'].apply(get_dates)

# take ranges and put each date into it's own column
# https://stackoverflow.com/questions/25559202/from-tuples-to-multiple-columns-in-pandas
new_cols = [
    'pre_launch_start_date', 
    'pre_launch_end_date', 
    'post_launch_start_date',
    'post_launch_end_date'
]

for i, col in enumerate(new_cols):
    games_df[col] = games_df['launch_dates'].apply(lambda dates: dates[i])

games_df = games_df.drop('launch_dates', axis=1)

In [41]:
# save to csv as game data has a lot of information
games_df.to_csv('./data/games.csv', index=False)

## Reddit Data

In [8]:
# connection to Redit API through praw
reddit = praw.Reddit(
    client_id=CLIENTID,
    client_secret=CLIENTSECRET,
    user_agent=USERAGENT
)

Version 7.0.0 of praw is outdated. Version 7.3.0 was released Thursday June 17, 2021.


### Submissions

For each game, get all Reddit submissions that were posted within the desired date range. Then save to a csv file that will be cleaned later.

In [43]:
game_submissions = []
for game in games_df[['name',
                      'pre_launch_start_date',
                      'pre_launch_end_date',
                      'post_launch_start_date',
                      'post_launch_end_date']].values:
    # need two separate calls (pre and post release date) as release date is
    # not included
    df_pre = get_submissions(game[0], game[1], game[2])
    df_pre['period'] = 'pre launch'
    game_submissions.append(df_pre)
    
    df_post = get_submissions(game[0], game[3], game[4])
    df_post['period'] = 'post launch'
    game_submissions.append(df_post)
    
submissions_df = pd.concat(game_submissions, ignore_index=True)
print(submissions_df.shape)
submissions_df.head()

(86657, 122)


,author,author_created_utc,author_flair_css_class,author_flair_text,author_fullname,created_utc,domain,edited,full_link,gilded,...,brand_safe,mod_reports,user_reports,updated_utc,category,steward_reports,removed_by,event_end,event_is_live,event_start
0,watisdisanywais,1.417940e+09,enclave,Enclave4Lyfe,t2_juza0,1.446595e+09,self.Fallout,1.446596e+09,https://www.reddit.com/r/Fallout/comments/3rf9...,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,fatyg5,1.380310e+09,text-desktop color-pcmr icon-steam,FX-8320@4.0GHz | R9 290 | 8GB,t2_db1a5,1.446595e+09,self.pcmasterrace,NaN,https://www.reddit.com/r/pcmasterrace/comments...,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Rodolfo619,1.358613e+09,None,None,t2_aabya,1.446595e+09,self.Fallout,NaN,https://www.reddit.com/r/Fallout/comments/3rfa...,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,INBREEDCLOWN,1.445405e+09,None,None,t2_rc9lq,1.446596e+09,self.fo4,NaN,https://www.reddit.com/r/fo4/comments/3rfai7/m...,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,pyjamapants8,1.391557e+09,None,None,t2_f4jkh,1.446596e+09,youtube.com,NaN,https://www.reddit.com/r/VGCovers/comments/3rf...,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
# check how many submissions per game
submissions_df['search_term'].value_counts()

Cyberpunk 2077                  27025
Fallout 4                       24446
DOOM Eternal                     8452
DOOM                             5679
Portal 2                         4533
BioShock Infinite                3807
XCOM® 2                          2401
Monster Hunter: World            1776
DARK SOULS™ III                  1226
Red Dead Redemption 2            1201
The Witcher® 3: Wild Hunt        1047
Total War: WARHAMMER              938
Hades                             847
They Are Billions                 543
Warhammer: Vermintide 2           480
Stardew Valley                    456
DARK SOULS™ II                    425
Total War: THREE KINGDOMS         423
Tropico 6                         337
Total War: WARHAMMER II           326
Bastion                           190
Valheim                            77
Sid Meier’s Civilization® VI       16
Sid Meier's Civilization® V         6
Name: search_term, dtype: int64

In [ ]:
submissions_df.to_csv('./data/submissions.csv', index=False)

### Comments & Authors

Getting every comment and every author for each comment in every submission and pushing them to an AWS rds instance.

In [5]:
# read in csv to prevent pending time on getting submissions
submissions_df = pd.read_csv('./data/submissions.csv')

C:\Users\Sam\anaconda3\envs\ga-dsir-capstone\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (12,21,33,36,37,39,40,43,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,70,71,72,74,75,77,79,81,82,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,116,117,118,120) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
# if there is error during data collection, get the next index after the
# last successful API call & write and restart data collection
last_id = '3rp9gm'
last_index = submissions_df[submissions_df['id'] == last_id].index[0] + 1
print(last_index)
submissions_df['id'][last_index:].values

1812


array(['3rmvk4', '3rmvk5', '3rmvli', ..., 'fog6ur', 'fog70a', 'fog72f'],
      dtype=object)

In [ ]:
save_comments_and_authors(submissions_df['id'][last_index:].values)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3rmvk4
0
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3rmvk5
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3rmvli
0
1
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3rmvor
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3rmvpc
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3rmvpq
0
Batch insert into 

1327
1328
1329
1330
1331
1332
1333
1334
1335
1336
1337
1338
1339
1340
1341
1342
1343
1344
1345
1346
1347
1348
1349
1350
1351
1352
1353
1354
1355
1356
1357
1358
1359
1360
1361
1362
1363
1364
1365
1366
1367
1368
1369
1370
1371
1372
1373
1374
1375
1376
1377
1378
1379
1380
1381
1382
1383
1384
1385
1386
1387
1388
1389
1390
1391
1392
1393
1394
1395
1396
1397
1398
1399
1400
1401
1402
1403
1404
1405
1406
1407
1408
1409
1410
1411
1412
1413
1414
1415
1416
1417
1418
1419
1420
1421
1422
1423
1424
1425
1426
1427
1428
1429
1430
1431
1432
1433
1434
1435
1436
1437
1438
1439
1440
1441
1442
1443
1444
1445
1446
1447
1448
1449
1450
1451
1452
1453
1454
1455
1456
1457
1458
1459
1460
1461
1462
1463
1464
1465
1466
1467
1468
1469
1470
1471
1472
1473
1474
1475
1476
1477
1478
1479
1480
1481
1482
1483
1484
1485
1486
1487
1488
1489
1490
1491
1492
1493
1494
1495
1496
1497
1498
1499
1500
1501
1502
1503
1504
1505
1506
1507
1508
1509
1510
1511
1512
1513
1514
1515
1516
1517
1518
1519
1520
1521
1522
1523
1524
1525
1526


Batch insert into reddit database.
Last updated submission id:  3rmwn0
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3rmwnp
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3rmwph
0
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3rmwpw
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3rmwtg
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3rmwuj
0
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3rmwzr
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3rmx40
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3rmx64
Batch insert into reddit da

Batch insert into reddit database.
Last updated submission id:  3rn03e
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3rn05t
0
1
2
3
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3rn08q
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3rn0fw
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3rn0hw
0
1
2
3
4
5
6
7
8
9
10
11
12
13
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3rn0it
0
1
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3rn0kz
0
1
2
Batch insert into reddit database.
Batch insert i

...

1369
1370
1371
1372
1373
1374
1375
1376
1377
1378
1379
1380
1381
1382
1383
1384
1385
1386
1387
1388
1389
1390
1391
1392
1393
1394
1395
1396
1397
1398
1399
1400
1401
1402
1403
1404
1405
1406
1407
1408
1409
1410
1411
1412
1413
1414
1415
1416
1417
1418
1419
1420
1421
1422
1423
1424
1425
1426
1427
1428
1429
1430
1431
1432
1433
1434
1435
1436
1437
1438
1439
1440
1441
1442
1443
1444
1445
1446
1447
1448
1449
1450
1451
1452
1453
1454
1455
1456
1457
1458
1459
1460
1461
1462
1463
1464
1465
1466
1467
1468
1469
1470
1471
1472
1473
1474
1475
1476
1477
1478
1479
1480
1481
1482
1483
1484
1485
1486
1487
1488
1489
1490
1491
1492
1493
1494
1495
1496
1497
1498
1499
1500
1501
1502
1503
1504
1505
1506
1507
1508
1509
1510
1511
1512
1513
1514
1515
1516
1517
1518
1519
1520
1521
1522
1523
1524
1525
1526
1527
1528
1529
1530
1531
1532
1533
1534
1535
1536
1537
1538
1539
1540
1541
1542
1543
1544
1545
1546
1547
1548
1549
1550
1551
1552
1553
1554
1555
1556
1557
1558
1559
1560
1561
1562
1563
1564
1565
1566
1567
1568


0
1
2
3
4
5
6
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3s11h3
0
1
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3s11hc
0
1
2
3
4
5
6
7
8
9
10
11
12
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3s11he
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
1

577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
792
793
794
795
796
797
798
799
800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
826


29
30
31
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3s15ph
0
1
2
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3s15rc
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3s15sb
0
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3s15se
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3s15t6
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3s167i
0
1
2
3
4
5
6
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3s16f1
0
1
2
3
4
5
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3s16fo

Batch insert into reddit database.
Last updated submission id:  3s1bvx
0
1
2
3
4
5
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3s1bwy
0
1
2
3
4
5
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3s1byz
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3s1c01
0
1
2
3
4
5
6
7
8
9
10
11
12
13
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3s1c21
0
1
2
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3s1c55
0
1
2
3
4
5
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3s1c6m
0
1
Batch insert into reddit database.
Batch insert into reddit database.
Last updated submission id:  3s1cco
0
1
2
3
Batch ins